In [ ]:
pip install duckduckgo-search newspaper3k lxml_html_clean requests

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 66.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 10.1 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=f330d3b1343c3f3c0c5432e75a28cd107ed466e1fec4bde9d9af04dc1ea1fde1
  Stored in directory: /root/.cache/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=d8f76d8df9496f076affd991f6e21500471a4a37774b69d44d3be7256930bc66
  Stored in 

In [ ]:
import requests
import json
import time
from newspaper import Article
from datetime import datetime

# ==========================================
# 1. CONFIGURATION (EDIT THIS)
# ==========================================

# 🔑 PASTE YOUR SERPER API KEY HERE
SERPER_API_KEY = "e34895fc6c85eb3738dcde0e0440be9c80687c15"

# 🎯 YOUR KEYWORDS
KEYWORDS = [
    "aadhaar enrollment decrease reasons",
    "aadhaar update pending issues",
    "aadhaar voter id link problems",
    "aadhaar authentication failure causes",
    "uidai server downtime history",
    "aadhaar biometric mismatch cases",
    "aadhaar address update rejected reasons"
]

# 🌐 OPTIONAL: LIMIT TO SPECIFIC SITES (Leave empty [] to search the whole web)
# Adding sites makes the search stricter but more relevant.
TARGET_SITES = [
    "thehindu.com",
    "indianexpress.com",
    "timesofindia.indiatimes.com",
    "livemint.com",
    "economictimes.indiatimes.com"
]

# ⚙️ SETTINGS
RESULTS_PER_KEYWORD = 20  # Max is 100 for Serper, but 10-20 is faster
OUTPUT_FILE = "aadhaar_large_dataset.json"

# ==========================================
# 2. HELPER FUNCTIONS
# ==========================================

def get_google_search_results(query, num_results):
    """
    Fetches search results from Serper.dev API.
    """
    url = "https://google.serper.dev/search"

    # Construct the query. If sites are defined, append them (OR logic)
    final_query = query
    if TARGET_SITES:
        site_operator = " OR ".join([f"site:{site}" for site in TARGET_SITES])
        final_query = f"{query} ({site_operator})"

    payload = json.dumps({
        "q": final_query,
        "num": num_results
    })

    headers = {
        'X-API-KEY': SERPER_API_KEY,
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status() # Raise error for bad responses (401, 403, 500)
        return response.json().get("organic", [])
    except Exception as e:
        print(f"   ⚠️ API Error: {e}")
        return []

def extract_article_details(url):
    """
    Uses Newspaper3k to download and parse the article text and date.
    """
    try:
        article = Article(url)
        article.download()
        article.parse()

        # Try to find the date
        pub_date = article.publish_date
        date_str = pub_date.strftime("%Y-%m-%d") if pub_date else "Unknown Date"

        return {
            "status": "success",
            "title": article.title,
            "date": date_str,
            "text_snippet": article.text[:500] # First 500 chars
        }
    except Exception as e:
        return {"status": "failed", "error": str(e)}

# ==========================================
# 3. MAIN EXECUTION LOOP
# ==========================================

def main():
    all_data = []
    total_keywords = len(KEYWORDS)

    print(f"🚀 Starting Large Scale Scrape...")
    print(f"🔑 API Key: {'*' * 10}{SERPER_API_KEY[-4:]}")
    print(f"📚 Keywords: {total_keywords}")
    print("-" * 40)

    for index, keyword in enumerate(KEYWORDS):
        print(f"\n[{index + 1}/{total_keywords}] Searching: '{keyword}'...")

        # 1. Get Links from API
        search_results = get_google_search_results(keyword, RESULTS_PER_KEYWORD)

        if not search_results:
            print("   ❌ No results found or API error.")
            continue

        print(f"   found {len(search_results)} links. Extracting content...")

        # 2. Process Each Link
        for result in search_results:
            link = result.get('link')

            # Skip if we already have this link (simple de-duplication)
            if any(d['url'] == link for d in all_data):
                continue

            # Extract content
            details = extract_article_details(link)

            if details['status'] == 'success':
                entry = {
                    "keyword": keyword,
                    "title": details['title'],
                    "url": link,
                    "event_date": details['date'], # Publish date as proxy
                    "snippet": result.get('snippet', ''), # Google's snippet
                    "full_content": details['text_snippet']
                }
                all_data.append(entry)
                # print(f"     ✅ {details['title'][:40]}... [{details['date']}]")
            else:
                # print(f"     ⚠️ Failed to parse: {link}")
                pass

    # ==========================================
    # 4. SAVE OUTPUT
    # ==========================================

    print("\n" + "=" * 40)
    print(f"🎉 Scraping Complete!")
    print(f"📄 Total Articles Saved: {len(all_data)}")

    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(all_data, f, indent=4, default=str)

    print(f"💾 Data saved to: {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

🚀 Starting Large Scale Scrape...
🔑 API Key: **********7c15
📚 Keywords: 7
----------------------------------------

[1/7] Searching: 'aadhaar enrollment decrease reasons'...
   found 10 links. Extracting content...

[2/7] Searching: 'aadhaar update pending issues'...
   found 10 links. Extracting content...

[3/7] Searching: 'aadhaar voter id link problems'...
   found 10 links. Extracting content...

[4/7] Searching: 'aadhaar authentication failure causes'...
   found 10 links. Extracting content...

[5/7] Searching: 'uidai server downtime history'...
   found 10 links. Extracting content...

[6/7] Searching: 'aadhaar biometric mismatch cases'...
   found 10 links. Extracting content...

[7/7] Searching: 'aadhaar address update rejected reasons'...
   found 10 links. Extracting content...

🎉 Scraping Complete!
📄 Total Articles Saved: 68
💾 Data saved to: aadhaar_large_dataset.json
